# Defining Recursive Functions in ACL2

Recall that functions in ACL2 are essentially equality statements:

    (double x) = (* 2 x)

Also recall that functions in ACL2 can have definitions that are given by cases, e.g.,

    (absolute-value x) = x,    if x >= 0
    (absolute-value x) = -x,   otherwise

Under some circumstances (more on this later), the right-hand side of the definition can actually use the function that is being defined.

This is not reason to panic. In fact, you are already familiar with many equations that have the same function in both sides of the equality. Here are just a few of these equations to remind you:

* $x^{n} = x \times x^{n-1}$
* $e^{x+y} = e^x e^y$
* $\sin(2\pi+x) = \sin(x)$

These equations are always true. Note that it doesn't matter how you define $x^n$, for example. The equations above should be true of your function. This is an important point. If nothing else, you can use those equations to test your implementation.

It just so happens that sometimes you can use these equations as *defining equations* to actually define the function you want. This is possible when your definition satisfies the ''Three Cs''.

## The Three Cs

1. Your equations must be **consistent**. This just means that if one equation says $f(3)=10$, another equation can't say that $f(3)=20$. Here are some equations that are  **not consistent**:

    (f x) = 1, if x is odd
    (f x) = 2, if x is positive
    (f x) = 1, if x is negative
    
Note that the first equation $f(5)=1$, but the second equation says that $f(5)=2$.

2. Your equations must be **complete**. This just means that the equations must cover all possible cases. For example, here are some equations that are **not complete**:

    (f x) = 1, if x is positive
    (f x) = 2, if x is negative

Notice these equations don't tell us what $f(0)$ should be, so they are not enough to define $f(x)$.

3. Your equations should be **computable**. This is trickier to get right at first. The idea is that if you write down the equations as a program, then the program will always terminate. Here is an equation that is **not computable**:

    (f x) = 1,          if x = 0
    (f x) = 1 + f(x-1), otherwise

This works some of the time. For example,

    f(3) = 1 + f(2) 
         = 1 + (1 + f(1)) 
         = 1 + (1 + (1 + f(0))) 
         = 1 + (1 + (1 + 0)) 
         = 1 + (1 + 1) 
         = 1 + 2 
         = 3
    
The reason this doesn't work is that, for example, there is no way to figure out what $f(-1)$ is:

    f(-1) = 1 + f(-2)
          = 1 + (1 + f(-3))
          = 1 + (1 + (1 + f(-4)))
          = ...???

That's not the only problem. For example, what is $f(2.5)$?

Informally, the equations $f$ is computable, if it satisfies these two properties:

* Every call to $f$ on the right-hand side of the equation must call $f$ on **smaller** arguments
* Eventually, the argument must hit a floor, e.g., 0, that doesn't have a smaller value

That's the general idea, or the intuition. It can be made much more precise, but to do so requires sophisticated mathematical ideas, namely *ordering relations* and *well-founded relations*. You typically learn about those in a course in Discrete Mathematics.

## Some Examples in ACL2

So let's look at some examples. Let's start with the function $triangular(n)$ that computes $1+2+\cdots+n$. Here are some equations that can be used to describe $triangular$:

    (triangular n) = 0,                             if n=0
    (triangular n) = (+ (triangular (- n 1)) n),    if n is a positive integer

Note that these equations must be true **no matter how you define f**. I.e., if you implement $triangular$ in some programming language, I should be able to check that $triangular(0)=0$ and that $triangular(n)=triangular(n-1)+n$ for any positive integer $n$. Again, this is a good way to test the function $triangular$, regardless of how you define it.

The question is, are these functions sufficient for defining $f$? Well, the definitions are trivially consistent. Since 0 is not a positive integer, every value of $n$ has only one equation that is appropriate, so there is no way to get two different values for $f(n)$.

However, the equations are not complete. There is no way, for example, to determine the value of $f(-3)$. We can fix that as follows:

    (triangular n) = 0,                             if n=0 or anything other than a positive integer
    (triangular n) = (+ (triangular (- n 1)) n),    if n is a positive integer

Our new equations are still consistent (because the two cases are still exclusive), and now they are complete. One of the equations will tell us the value of $triangular(n)$ for any $n$.

Moreover, these equations are computable! Notice that each call on the right-hand side invokes $f$ with a smaller value, i.e., $n-1 < n$. And eventually, we get to the $triangular(0)$ which doesn't depend on the value of any smaller $triangular(n)$.

We are now ready to define this function in ACL2. We only need to know a way to check whether "n=0 or anything other than a positive integer". This is such a common property that ACL2 defines a built-in function just for this purpose. `(zp n)` is true precisely when "n=0 or anything other than a positive integer".

The definition of `triangular` in ACL2 is as follows:

In [ ]:
(defsnapshot triangular-definition)

(definec triangular (n :nat) :nat
  (if (zp n)
      0
      (+ (triangular (- n 1)) n)))

Now let's look at a famous function. The Riemann zeta function is defined as

$$\zeta(s) = \sum_{k=1}^\infty \frac{1}{s^2}$$

That is an infinite sum, but for any given value of $s$ we can approximate $zeta(s)$ by adding up only the first $n$ terms. For example, we can approximate $\zeta(2)$ by adding up the first three terms:

$$\zeta(2) \approx \frac{1}{1^2} + \frac{1}{2^2} + \frac{1}{3^2}$$

We get a better approximation by adding up the first 10 terms:

$$\zeta(2) \approx \frac{1}{1^2} + \frac{1}{2^2} + \cdots + \frac{1}{10^2}$$

What we really want, of course, is to add up the first $n$ terms. We use the notation $\zeta_2(n)$ to mean the sum of the first $n$ terms of $\zeta(2)$:

$$\zeta(2) \approx \zeta_2(n) = \frac{1}{1^2} + \frac{1}{2^2} + \cdots + \frac{1}{n^2}$$

To define `zeta2` in ACL2, we consider the defining equations. We are adding the first $n$ terms of the form $\frac{1}{k^2}$. This is very similar to the previous example, except each term is a little more complex. With a little thought you can see that these defining equations should do the trick:

    (zeta2 n) = 0,                                    if n=0 or anything other than a positive integer
    (zeta2 n) = (+ (zeta2 (- n 1)) (/ 1 (* n n))),    if n is a positive integer

Once you have the defining equations, it's easy to write down the definition in ACL2 syntax:


In [ ]:
(defsnapshot zeta2-definition)

(definec zeta2 (n :nat) :rational
  (if (zp n)
      0
      (+ (zeta2 (- n 1)) 
         (/ 1 (* n n)))))

The key lesson is this: Once you have the defining equations, writing down the definition in ACL2 is easy. So if you find yourself struggling to program in ACL2, focus on the defining equations. Do not start by writing down `(definec f ...)`! It pays off to think through the defining equations first. In another tutorial, we'll look at a method for discovering these equations. For now, get some practice defining some functions that follow the same pattern as `triangular` and `zeta2`. 

## Exercises

Write functions in ACL2 to compute the following values:

* $\text{factorial}(n) = 1 \times 2 \times \dots \times n$
* $\text{harmonic}(n) = \frac{1}{1} + \frac{1}{2} + \dots + \frac{1}{n}$
* $\text{alternating-harmonic}(n) = \frac{1}{1} - \frac{1}{2} + \dots + (-1)^{n+1} \frac{1}{n}$
* $\text{sum-squares}(n) = 1^2 + 2^2 + \cdots + n^2$
* $\text{sum-powers-of-2}(n) = 2^0 + 2^1 + 2^2 + \cdots + 2^n$

Hint: To compute $x^n$, use the built-in ACL2 function `(expt x n)`.

Hint: Don't forget to start with `defsnapshot` so you can more easily debug your code!